# Image Processing Project - NYC Detective
Carlos Ponce (`cmp279`)  
Zachary Hunt (`zh362`)  
Mykyta Turpitka (`mt689`)

# Imports

In [ ]:
import os
import keras
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import io
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import img_to_array, load_img

## A - Data Loading

In [ ]:
input_directory = "./data/"
photo_directory = input_directory + "processed/"
with open(input_directory + "PhotoTable.p", 'rb') as pickle_file:
    photo_info = pickle.load(pickle_file)
photo_info

In [ ]:
def prepare_data(list_of_images):
    x = [] # images as arrays
    for image in list_of_images:
        x.append(img_to_array(load_img(image,target_size=sample_photo.shape)))
    return np.array(x)

In [ ]:
train = photo_directory
training = [train+i for i in os.listdir(train)]
sample_photo = io.imread(training[0])
image_shape = (124, 187, 1)
train_X_raw = prepare_data(training)
train_Y_raw = photo_info["Target"]

In [ ]:
# (train_X_raw, train_Y_raw), (test_X_raw, test_Y_raw) = mnist.load_data()
print("Train shapes:", train_X_raw.shape, "->", train_Y_raw.shape)
# print("Test shapes:", test_X_raw.shape, "->", test_Y_raw.shape)

In [ ]:
plt.imshow(train_X_raw[0, :, :, 0])

## B - Preprocessing

In [ ]:
# I chose not to use a function
# Reshape to single channel, scale down to [0, 1]
train_X = train_X_raw[:, :, :, :1] / 255.0
# train_Y = to_categorical(train_Y_raw)
train_Y = train_Y_raw

## C - CNN Implementation

In [ ]:
def create_cnn(dropout=False, conv_layer2=False, learning_rate=0.001):
    # Define using Sequential
    model = Sequential()
    # Convolution Layer
    model.add(
        Conv2D(
            32,
            (3, 3),
            activation="relu",
            kernel_initializer="he_uniform",
            input_shape=image_shape,
        )
    )
    # Maxpooling Layer
    model.add(MaxPooling2D((2, 2)))
    
    if conv_layer2:
        # Convolution Layer
        model.add(
            Conv2D(
                64,
                (3, 3),
                activation="relu",
                kernel_initializer="he_uniform",
                input_shape=image_shape,
            )
        )
        # Maxpooling Layer
        model.add(MaxPooling2D((2, 2)))
    
    # Flatten Output
    model.add(Flatten())
    # Droupout Layer
    if dropout:
        model.add(Dropout(0.5))
    # Dense Layer of 100 neurons
    model.add(Dense(100, activation="relu", kernel_initializer="he_uniform"))
    model.add(Dense(1, activation="softmax"))
    # Initialize Optimizer
    opt = RMSprop(lr=learning_rate)  # , momentum=0.9)
    # Compile Model
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

    return model

## D - Training and Evaluating CNN

In [ ]:
train_Y.shape

In [ ]:
train_X.shape

In [ ]:
model = create_cnn(conv_layer2=True)  # dropout=True, learning_rate=1e-06)
model.fit(train_X, train_Y, batch_size=32, epochs=10, validation_split=0.1)
# score = model.evaluate(test_X, test_Y, verbose=0)

In [ ]:
image_shape

## E - Experimentation

In [ ]:
train_X[:5].shape

In [ ]:
model.predict(train_X)